In [1]:
from datasets import load_dataset,load_from_disk,Dataset
import pandas as pd

In [2]:
# 导入训练与测试数据集
# train_dataset_df = pd.read_parquet('datasets/alpaca-0.2-train') # datasets/alpaca-0.2-train
# test_dataset_df = pd.read_parquet('datasets/alpaca-0.2-test')  # datasets/alpaca-0.2-test
datasets_name = 'alpaca' # 'alpaca'/'gptrqa'

if datasets_name == 'alpaca':
    train_dataset_df = load_from_disk('datasets/alpaca-0.2-train') # datasets/alpaca-0.2-train
    test_dataset_df = load_from_disk('datasets/alpaca-0.2-test')   # datasets/alpaca-0.2-test
else:
    train_dataset_df = load_from_disk('datasets/GPTRQA-train') # datasets/alpaca-0.2-train
    test_dataset_df = load_from_disk('datasets/GPTRQA-test')   # datasets/alpaca-0.2-test

In [3]:
test_size=[0.0,0.2,0.5,0.8]
DI_generation_texts_pth = [
    f'ablation/GenText/{datasets_name}/DI_{datasets_name}_gen_{ts}.txt' for ts in test_size
]

In [4]:
contents = []
for path in DI_generation_texts_pth:
    with open(path, 'r', encoding='utf-8') as file:
        contents.append(file.readlines())

In [5]:
refs = [ [i.replace('enquiry: ',"")] for i in test_dataset_df['target_text']]
print(refs[0])
print(contents[0][0])
print(contents[1][0])
print(contents[2][0])
print(contents[3][0])

['Generate a simile comparing a sunset to fiction. ']
Create a simile for a sunset.

Create a metaphor to describe a sunset.

Create a metaphor that compares a sunset to a fiction piece.

Generate a metaphor to describe a sunset.



In [6]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
#emb = model.encode("A sentence about cats.", convert_to_tensor=True)

In [7]:
from tqdm import tqdm

scores=[]
for i in tqdm(range(len(contents[0]))):
    score=0.0
    base_emd=model.encode(refs[i])
    for j in range(len(contents)):
        each_emd=model.encode(contents[j][i])
        cosine_sim = util.pytorch_cos_sim(base_emd, each_emd)
        score=score+cosine_sim
    scores.append(score.item())

100%|██████████| 10401/10401 [03:08<00:00, 55.26it/s]


In [8]:
import numpy as np
k = 2000
scores=np.array(scores)
topk_indices = scores.argsort()[-k:][::-1]
topk_values = scores[topk_indices]

# print("Top-k 索引:", topk_indices.tolist())
# print("Top-k 值:", topk_values.tolist())

selected_idx=[]

for i in range(len(topk_indices)):
    if topk_values[i]<3.8 and topk_values[i]>3.7:
        selected_idx.append(topk_indices[i].item())

In [9]:
for idx in selected_idx:
    print(refs[idx])
    for i in range(len(contents)):
        print(contents[i][idx])

['What artist sang the song "Can\'t Stop the Feeling"? ']
Identify the artist who sang the song "Can't stop the feeling".

Name the artist who sang the song "Can't Stop the Feeling".

Who is the artist who sang the song "Can't stop the feeling"?

Name the artist who sang the song "Can't Stop the Feeling".

['How does artificial intelligence assist in medical diagnosis? ']
How can artificial intelligence assist in medical diagnosis?

Explain how artificial intelligence can help medical diagnosis.

Explain how artificial intelligence can help medical diagnosis.

Explain how artificial intelligence can assist in medical diagnosis.

['Name one downside of becoming a freelancer ']
Identify a potential downside of becoming a freelancer.

Identify a potential downside of becoming a freelancer.

Identify a potential downside of becoming a freelancer.

Name a potential downside of becoming a freelancer.

['What is the purpose of incorporating a feedback system in a machine learning algorithm? '